In [1]:
from utils import *
from sklearn.model_selection import train_test_split

df = pd.read_excel('data/DS_assessment.xlsx', sheet_name = 'Data')
df.shape

(5000, 13)

Standardize column names to lowercase, with spaces denoted by _

In [ ]:
df = standardize_column_names(df, custom_mapping: dict = {'CCAvgSpending': 'cc_avg_spending', 'InternetBanking': 'internet_banking'})
df.columns

## Perform Sanity Checks

### Check for missing values in the datasets

Check if the dataset contains any missing values in its columns.

In [2]:
check_missing_columns(df)

,contains_missing,missing_count,missing_percent
ID,False,0,0.00
Age,False,0,0.00
Experience,False,0,0.00
Income,True,20,0.40
Postal Code,False,0,0.00
Family Size,True,9,0.18
CCAvgSpending,False,0,0.00
Education,False,0,0.00
Mortgage,False,0,0.00
Investment Account,False,0,0.00


Income and Family Size columns contain missing values. Given that the number and percentage of records with missing Income (0.4%) or Family Size (0.18%) values is very small and insignificant, let's drop these records.

In [4]:
df.dropna(inplace = True)
df.shape

(4971, 13)

## Perform Data Preprocessing

### Preprocessing Steps:

1) Cast Postal Code to categorical datatype.
2) Convert InternetBanking and Personal Loan to boolean integers (i.e. 0 = NO. 1 = YES).
3) Handle outlier/erroneous records.

In [12]:
df['Postal Code'] = df['Postal Code'].astype('category')
df.dtypes

ID                       int64
Age                      int64
Experience               int64
Income                 float64
Postal Code           category
Family Size            float64
CCAvgSpending          float64
Education               object
Mortgage                 int64
Investment Account       int64
Deposit Account          int64
InternetBanking         object
Personal Loan           object
dtype: object

In [13]:
mapping = {'NO': 0, 'YES': 1}

df['InternetBanking'] = df['InternetBanking'].map(mapping)
df['Personal Loan'] = df['Personal Loan'].map(mapping)
df.dtypes

ID                       int64
Age                      int64
Experience               int64
Income                 float64
Postal Code           category
Family Size            float64
CCAvgSpending          float64
Education               object
Mortgage                 int64
Investment Account       int64
Deposit Account          int64
InternetBanking          int64
Personal Loan            int64
dtype: object

Let's take a look at the summary statistics of the data to detect any potential outliers or errorneous data.

In [14]:
df.describe()

,ID,Age,Experience,Income,Family Size,CCAvgSpending,Mortgage,Investment Account,Deposit Account,InternetBanking,Personal Loan
count,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000,4971.000000
mean,2505.909877,45.334339,20.100583,73.805472,2.394689,1.937556,56.626836,0.104607,0.060350,0.596862,0.096359
std,1439.973439,11.458282,11.462613,46.074179,1.147091,1.746019,101.813329,0.306077,0.238158,0.490577,0.295112
min,1.000000,23.000000,-3.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1261.500000,35.000000,10.000000,39.000000,1.000000,0.700000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2506.000000,45.000000,20.000000,64.000000,2.000000,1.500000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,3750.500000,55.000000,30.000000,98.000000,3.000000,2.500000,101.000000,0.000000,0.000000,1.000000,0.000000
max,5000.000000,67.000000,43.000000,224.000000,4.000000,10.000000,635.000000,1.000000,1.000000,1.000000,1.000000


It could be observed that the minimum value for Experience is -3. This does not make any sense, as it is not possible for anyone to have negative years of professional experience. Let's dive deeper to see which are the records with negative Experience.

In [23]:
neg_exp = df[df.Experience < 0]
print(neg_exp.shape)
neg_exp

(51, 13)


,ID,Age,Experience,Income,Postal Code,Family Size,CCAvgSpending,Education,Mortgage,Investment Account,Deposit Account,InternetBanking,Personal Loan
89,90,25,-1,113.0,94303,4.0,2.30,Advanced Degree,0,0,0,0,0
226,227,24,-1,39.0,94085,2.0,1.70,Graduate,0,0,0,0,0
315,316,24,-2,51.0,90630,3.0,0.30,Advanced Degree,0,0,0,1,0
451,452,28,-2,48.0,94132,2.0,1.75,Advanced Degree,89,0,0,1,0
524,525,24,-1,75.0,93014,4.0,0.20,Undergrad,0,0,0,1,0
536,537,25,-1,43.0,92173,3.0,2.40,Graduate,176,0,0,1,0
540,541,25,-1,109.0,94010,4.0,2.30,Advanced Degree,314,0,0,1,0
576,577,25,-1,48.0,92870,3.0,0.30,Advanced Degree,0,0,0,0,0
583,584,24,-1,38.0,95045,2.0,1.70,Graduate,0,0,0,1,0
597,598,24,-2,125.0,92835,2.0,7.20,Undergrad,0,1,0,0,0


There are 51 records with negative Experience. Percentage wise, these records account for 1.03% (i.e. 51/4971) of the records, which is a very small insignificant proportion. Hence, let's drop these records as well.

In [27]:
df =  df[df.Experience >= 0]
df.shape

(4920, 13)

## Perform Feature Engineering 

### Features to create

1. income_cc_spending_diff: Difference between annual income and annual credit card spending (i.e. income - 12 x cc_avg_spending)
2. cc_spending_income_ratio: Ratio between annual credit card spending and annual income (i.e. 12 x cc_avg_spending / income)
3. mortage_income_diff: Difference between home mortgage value and annual income and  (i.e. mortage - income)
4. mortage_income_ratio: Ratio between home mortgage value and annual income (i.e. mortage / income)

In [30]:
df['income_cc_spending_diff'] = df['income'] - 12 * df['cc_avg_spending']
df['cc_spending_income_ratio'] = 12 * df['cc_avg_spending'] / df['income']

df['mortage_income_diff'] = df['mortgage'] - df['income']
df['mortage_income_ratio'] = df['mortgage'] / df['income']

df.head(1)

/tmp/ipykernel_13281/4181727763.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13281/4181727763.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_13281/4181727763.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_1

## Perform Exploratory Data Analysis

### Analyse the distribution of target variable, personal_loan

In [39]:
plot_categories_distribution(df, 'personal_loan', width = 800, height = 400)

It could be observed that the dataset is imbalanced in terms of the target variable, personal_loan. The number of customers that do not take up the personal loan (i.e. personal_loan = 0) is more than 9x the number of customers that take up the loan (i.e. personal_loan = 1). In such a scenario, accuracy might not be the most appropriate evaluation metric to use, as it is biased towards the majority class (i.e. personal_loan = 0). Instead, f1 score might be a more appropriate evaluation metric to use, given that it is the harmonic mean of precision and recall, and hence unbiased towards either of the classes.

In [66]:
loan_probs = compute_loan_probability(df, category_col = 'postal_code')
loan_probs.head()

,Postal Code,Personal Loan,Probability
819,95135,1,0.666667
921,96008,1,0.666667
501,93311,1,0.500000
723,94705,1,0.500000
289,92056,1,0.500000
...,...,...,...
417,92694,1,0.000000
415,92692,1,0.000000
413,92691,1,0.000000
407,92673,1,0.000000


In [ ]:
loan_probs.tail()

In [69]:
plot_category_loan_distribution(df, category_col = 'education')

In [70]:
plot_category_loan_distribution(df, category_col = 'investment_account', height = 400)

In [71]:
plot_category_loan_distribution(df, category_col = 'deposit_account', height = 400)

In [72]:
plot_category_loan_distribution(df, category_col = 'internet_banking', height = 400)

In [73]:
px.box(df, x = 'Age', color = 'Personal Loan')

In [74]:
px.box(df, x = 'Experience', color = 'Personal Loan')

In [75]:
px.box(df, x = 'Income', color = 'Personal Loan')

In [76]:
px.box(df, x = 'Family Size', color = 'Personal Loan')

In [77]:
px.box(df, x = 'CCAvgSpending', color = 'Personal Loan')

In [78]:
px.box(df, x = 'Mortgage', color = 'Personal Loan')

In [79]:
px.box(df, x = 'income_cc_spending_diff', color = 'Personal Loan')

In [80]:
px.box(df, x = 'cc_spending_income_ratio', color = 'Personal Loan')

In [81]:
px.box(df, x = 'mortage_income_diff', color = 'Personal Loan')

In [82]:
px.box(df, x = 'mortage_income_ratio', color = 'Personal Loan')

## Encode categorical features into numerical values

In [ ]:
df, encoder_dict = encode_categorical_features(df)

## Perform stratified train test split based on target variable, personal_loan

TODO:

2) Encode categorical features: Postal Code, Education, Investment Account, Deposit Account, InternetBanking
3) Perform stratified train test split based on labels
4) modelling

In [87]:
def rename_columns(df: pd.DataFrame, custom_mapping: dict = {'CCAvgSpending': 'cc_avg_spending', 'InternetBanking': 'internet_banking'}):
    return df.rename(
        columns = lambda x: x.lower().replace(' ', '_') if x not in custom_mapping else custom_mapping[x]
    )

In [88]:
rename_columns(df)

,id,age,experience,income,postal_code,family_size,cc_avg_spending,education,mortgage,investment_account,deposit_account,internet_banking,personal_loan,income_cc_spending_diff,cc_spending_income_ratio,mortage_income_diff,mortage_income_ratio
0,1,25,1,49.0,91107,4.0,1.6,Undergrad,0,1,0,0,0,29.8,0.391837,-49.0,0.000000
1,2,45,19,34.0,90089,3.0,1.5,Undergrad,0,1,0,0,0,16.0,0.529412,-34.0,0.000000
2,3,39,15,11.0,94720,1.0,1.0,Undergrad,0,0,0,0,0,-1.0,1.090909,-11.0,0.000000
4,5,35,8,45.0,91330,4.0,1.0,Graduate,0,0,0,0,0,33.0,0.266667,-45.0,0.000000
5,6,37,13,29.0,92121,4.0,0.4,Graduate,155,0,0,1,0,24.2,0.165517,126.0,5.344828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40.0,92697,1.0,1.9,Advanced Degree,0,0,0,1,0,17.2,0.570000,-40.0,0.000000
4996,4997,30,4,15.0,92037,4.0,0.4,Undergrad,85,0,0,1,0,10.2,0.320000,70.0,5.666667
4997,4998,63,39,24.0,93023,2.0,0.3,Advanced Degree,0,0,0,0,0,20.4,0.150000,-24.0,0.000000
4998,4999,65,40,49.0,90034,3.0,0.5,Graduate,0,0,0,1,0,43.0,0.122449,-49.0,0.000000
